In [108]:
import pandas as pd
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
import pandas_ta as ta
import http.client, urllib.parse
import json
from datetime import datetime
import time
from keras.models import load_model
import pickle
from sklearn.decomposition import PCA
import numpy as np

In [109]:
#import all the data
apple = pd.read_csv('../data/test_AAPL.csv')
tesla = pd.read_csv('../data/test_TSLA.csv')
microsoft = pd.read_csv('../data/test_MSFT.csv')
google = pd.read_csv('../data/test_GOOG.csv')

In [110]:
#get the array of opening and closing price
apple_opening_prices = apple['Open']
apple_closing_prices = apple['Close']
tesla_opening_prices = tesla['Open']
tesla_closing_prices = tesla['Close']
microsoft_opening_prices = microsoft['Open']
microsoft_closing_prices = microsoft['Close']
google_opening_prices = google['Open']
google_closing_prices = google['Close']

In [111]:
#trend is 1 for upward trend and 0 for downward trend
#apple
apple_trend = []
for i in range(0,len(apple_opening_prices)):
    if apple_opening_prices[i]>apple_closing_prices[i]:
        apple_trend.append(0)
    else:
        apple_trend.append(1)

#google
google_trend = []
for i in range(0,len(google_opening_prices)):
    if google_opening_prices[i]>google_closing_prices[i]:
        google_trend.append(0)
    else:
        google_trend.append(1)
        
#tesla
tesla_trend = []
for i in range(0,len(tesla_opening_prices)):
    if tesla_opening_prices[i]>tesla_closing_prices[i]:
        tesla_trend.append(0)
    else:
        tesla_trend.append(1)

#microsoft
microsoft_trend = []
for i in range(0,len(microsoft_opening_prices)):
    if microsoft_opening_prices[i]>microsoft_closing_prices[i]:
        microsoft_trend.append(0)
    else:
        microsoft_trend.append(1)


In [133]:
#features to be used
indicators = ['High','Low','Open','Volume','Adj Close','H-L','O-C','5MA',
              '10MA','20MA','7SD','EMA8','EMA21','EMA34','EMA55','RSI_14','Sentiment']


#function to calculate the percentages of right trend prediction
def calculate_score(ticker_name,stock_data,model_path,x_normaliser,y_normaliser,trend,model_name,opening_prices):
    #load model
    if model_name != 'random forest':
        model = load_model(model_path)
    else:
        model = pickle.load(open(model_path, 'rb'))
    
    #load the scalers
    with open(x_normaliser, "rb") as input_file:
                x_scaler = pickle.load(input_file)

    with open(y_normaliser, "rb") as input_file:
                y_scaler = pickle.load(input_file)

    data = stock_data[indicators]
    scaled_data = x_scaler.transform(data)
    
    pca = PCA(n_components=4)
    scaled_data = pca.fit_transform(scaled_data)
    
    
    if model_name=='lstm' or model_name=='gru':
        scaled_data = np.expand_dims(scaled_data, axis=1)
        
    pred = model.predict(scaled_data)
    pred = y_scaler.inverse_transform(pred.reshape(-1,1))

    temp = []
    for i in range(0,len(pred)):
        if opening_prices[i]>pred[i]:
            temp.append(0)
        else:
            temp.append(1)
    
    ct = 0

    for i in range(0,len(temp)):
        if temp[i]==trend[i]:
            ct=ct+1
    
    percentage = (ct/len(temp))*100
    
    print('The trend accuracy of the model for ' + ticker_name + ' ' + model_name + ' is ' + str(percentage)) 
    

In [134]:
#apple
calculate_score('AAPL',apple,'../models/AAPL_ann.h5','../normalizers/scaler_x_AAPL_ann.pkl',
                '../normalizers/scaler_y_AAPL_ann.pkl',apple_trend,'ann',apple_opening_prices)

calculate_score('AAPL',apple,'../models/AAPL_lstm.h5','../normalizers/scaler_x_AAPL_ann.pkl',
                '../normalizers/scaler_y_AAPL_ann.pkl',apple_trend,'lstm',apple_opening_prices)

calculate_score('AAPL',apple,'../models/AAPL_gru.h5','../normalizers/scaler_x_AAPL_ann.pkl',
                '../normalizers/scaler_y_AAPL_ann.pkl',apple_trend,'gru',apple_opening_prices)

calculate_score('AAPL',apple,'../models/random_forest_AAPL.sav','../normalizers/scaler_x_AAPL_ann.pkl',
                '../normalizers/scaler_y_AAPL_ann.pkl',apple_trend,'random forest',apple_opening_prices)







2/2 [==============================] - 0s 12ms/step
The trend accuracy of the model for AAPL ann is 51.515151515151516


/Users/varun/miniforge3/envs/guided_study/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
2022-07-17 00:35:38.308141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/varun/miniforge3/envs/guided_study/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


2/2 [==============================] - 0s 30ms/step
The trend accuracy of the model for AAPL lstm is 51.515151515151516


2022-07-17 00:35:38.538908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/varun/miniforge3/envs/guided_study/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


2/2 [==============================] - 0s 37ms/step
The trend accuracy of the model for AAPL gru is 51.515151515151516
The trend accuracy of the model for AAPL random forest is 48.484848484848484


2022-07-17 00:35:38.830652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/varun/miniforge3/envs/guided_study/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


array([[1046.4847 ],
       [1041.188  ],
       [1037.553  ],
       [1037.5525 ],
       [1036.1464 ],
       [1040.112  ],
       [1041.8748 ],
       [1035.025  ],
       [1034.9725 ],
       [1044.0548 ],
       [1076.4565 ],
       [1073.2604 ],
       [1065.489  ],
       [1059.4875 ],
       [1050.2865 ],
       [1046.1843 ],
       [1028.2363 ],
       [ 993.26324],
       [ 966.6555 ],
       [ 932.93396],
       [ 883.77545],
       [ 848.73254],
       [ 816.94464],
       [ 796.4535 ],
       [ 788.34753],
       [ 784.45514],
       [ 770.47095],
       [ 777.0292 ],
       [ 776.4294 ],
       [ 783.97015],
       [ 818.8758 ],
       [ 820.5118 ],
       [ 797.9846 ]], dtype=float32)